In [ ]:
import sys
import serial
import time
import pyqtgraph as pg   
import numpy as np
import mysql.connector as mysql
from datetime import datetime
from PyQt5 import uic
from PyQt5.QtWidgets import QMainWindow, QApplication, QMessageBox, QVBoxLayout, QTableWidgetItem
from PyQt5.QtCore import QTimer
from scipy import signal

class ArduinoApp(QMainWindow):
    def __init__(self):
        super().__init__()

        uic.loadUi('integracion.ui', self) # Cargar Interfaz 
        
        # Configurar el puerto serial
        try:
            self.arduino = serial.Serial('COM7', 9600, timeout=1)  # Verificar el COM correcto
            time.sleep(2)  # Esperar a que Arduino reinicie
            self.arduino.flush()  # Limpiar buffer de entrada
        except Exception as e:
            QMessageBox.critical(self, 'Error', f'No se pudo conectar al Arduino: {e}')
            sys.exit()
        
        # Conectar Widgets
        # self.WIDGETTTT.connect(self.send_temp)
        # self.WIDGETTTT.connect(self.send_power)
        # self.WIDGETTTT.connect(self.initial_parameters)

    # Funciones

    # Lee datos del Arduino
    def read_data(self):
        try:
            temperatura = self.arduino.readline().decode('utf-8', errors='ignore').rstrip()

    # Envía el valor de la potencia de 0 a 100 para la bombilla
    def send_power_bulb(self, power):
        if self.arduino and self.arduino.is_open:
            self.arduino.write(f'P{power}\n'.encode('utf-8')) #HAY QUE APLICAR UN WIDGET
    
    # Envía el valor de la potencia de 0 a 100 para el ventilador
    def send_power_fan(self, power):
        if self.arduino and self.arduino.is_open:
            self.arduino.write(f'F{power}\n'.encode('utf-8'))

    # Función para exportar datos a CSV
    def csv_export(self):
        try:
            with open('datos.csv', 'w', newline='') as csvfile:
                writer = csv.writer(csvfile)
                writer.writerow(['Temperatura', 'Potencia', 'Fecha'])
                for fila in self.datos:
                    writer.writerow(fila)
            QMessageBox.information(self, 'Éxito', 'Datos exportados a datos.csv')
        except Exception as e:
            QMessageBox.critical(self, 'Error', f'No se pudo exportar los datos: {e}')

        #PARA POP-UP DE DESCONEXIÓN DEL CABLE 
        except serial.SerialException as e:
            QMessageBox.critical(self, 'Error de conexión', f'Se perdió la conexión con el Arduino: {e}')
            self.parar_dato()  # Detener la adquisición de datos
            self.boton_iniciar.setEnabled(False)  # Deshabilitar el botón de inicio
            if hasattr(self, 'arduino') and self.arduino.is_open:
                self.arduino.close()  # Cerrar el puerto serial si está abierto

    def closeEvent(self, event):
        # Cerrar el puerto serial al cerrar la aplicación
        if self.arduino.is_open:
            self.arduino.close()